In [1]:
import os, re
os.chdir("/Users/Pace/Documents/Education/Universität Potsdam/Semester 4 - Summer 2024/Evaluating Chatp-Optimized Language Models/Project")
from backends import ModelSpec, Model, get_model_for, load_model_registry

In [2]:
class Game:
    def __init__(self, llm: Model, system_prompt: str, instructions: str) -> None:
        self.llm: Model = llm
        self.system_prompt: str = system_prompt
        self.instructions: str = instructions
        
        # Dynamically builds the board according to number of fields specified
        self.n_fields: int = 23
        cells: list = []
        for _ in range(self.n_fields):
            cells.append("□")
        self.board = " ".join(cells).rstrip()

        self.rolls: list = [3, 6, 4, 5, 2, 6, 4, 1] # TODO replace with dynamic solution

        self.turn: int = 0 # turn counter
        self.current_position: int = 0 # TODO replace with dynamic solution

        self.context: list = [] # memorized context
        self.current_state: str = self.board # current state of the board
        self.prefix: str = "MY MOVE:" # tag used for parsing
        self.sign: str = " -> "
        self.token_out: bool = False # false if no token on board yet

    def make_move(self) -> None:
        # Creates user message
        match self.turn:
            case 0:
                initial_message: str = f"Are you ready to play?\nBeggining State: {self.board}\nTurn Number: {self.turn}, Roll: {self.rolls[self.turn]}.\nWhere will you move your token? Let's think step by step."
                message: str = self.instructions + "\n" + initial_message

            case _:
                message: str = f"Current State: {self.current_state}\nNext Turn Number: {self.turn}, Roll: {self.rolls[self.turn]}.\nWhere will you move your token? Let's think step by step."
        
        self.add_message(message, role="user")

        # Gathers and parses the assistant's move
        _, _, response_text = self.llm.generate_response(self.context)
        move: str = self.parse_reply(response_text)

        if self.check_move(move, self.rolls[self.turn]):
            print(self.current_state)

            self.current_position = move
            self.add_message(response_text, role="assistant")
            self.turn += 1

            match self.turn:
                case 1:
                    print(move)

                    if move == 6:
                        self.update_board(move)
                        self.token_out = True
                        self.last_response = response_text

                case _ if self.turn > 1:
                    self.token_out = not self.token_out and self.rolls[self.turn] == 6

                    if self.token_out:
                        self.update_board(move)

                    print(self.current_state)

        else:
            print(f"Fail at turn: {self.turn}")
            print(f"Roll: {self.rolls[self.turn]}")
            print(response_text)
            
            match self.turn:
                case 0:
                    self.last_response = response_text
                case 1:
                    print(f"Move: {move}")

            print(self.current_state)

    def check_move(self, move: str, roll: int) -> bool:
        if not self.token_out:
            return (roll == 6 and move == 1) or (roll != 6 and move == 0)

        else:
            if move > self.n_fields:
                print("F1")
                return False
            
            if self.current_position > move:
                print("F2")
                return False

            if self.current_position > move:
                return True

            else:
                print(f"Fails: {roll + self.current_position} != {move}")
                return False

    def add_message(self, message: str, role: str = "user") -> None:
        if self.context == []:
            self.context = [{"role": "system", "content": self.system_prompt}]

        self.context.append({"role": role, "content": message})

    def parse_reply(self, text: str) -> int:
        parsed_reply: list = text.split(self.prefix)
        print(parsed_reply[1])

        final = parsed_reply[1].split(self.sign)
        print(f'Final: {final}')

        return int(final[1])

    def update_board(self, move: int) -> None:
        b = self.board
        split_b = b.split()
        split_b[move-1] = 'X'
        new_b = ' '.join(split_b)
        self.current_state = new_b

    def reset(self) -> None:
        self.turn = 0
        self.context = []

In [3]:
load_model_registry()
THIS_MODEL: dict = dict(model_id="gpt-3.5-turbo-1106", backend="openai", model_name="gpt-3.5-turbo-1106")
llm: Model = get_model_for(THIS_MODEL)
llm.set_gen_args(temperature=0.0, max_tokens=400)

In [4]:
with open('games/ludo/resources/initial_prompts/simple_prompt_v1.txt', 'r') as f:
    system_prompt = f.read()

with open('games/ludo/resources/initial_prompts/no_board_prompt.txt', 'r') as f:
    content = f.read()

instance: Game = Game(llm, content, system_prompt)

In [9]:
instance.make_move()

 X -> 5
Final: [' X', '5']
Fail at turn: 2
Roll: 4
There are 22 empty fields and 1 occupied field. In total: 22 + 1 = 23 fields. My token is on field number 1. You have rolled 4. Since 1 + 4 = 5, I need to move my token to field number 5. This is 4 fields away from my previous position.
MY MOVE: X -> 5
□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □


In [10]:
for turn_dict in instance.context:
    print(f"Role: {turn_dict['role']}")
    print(turn_dict["content"] + "\n")

Role: system
Welcome to ludogame! Your task is to move a from start to end across the board. I will give you a board with empty fields. An empty field is marked like this □.
Your token is marked with X. You token X counts as an occupied field. The total number of fields remains unchanged throughout the game.

General instructions:
1) Every turn I will give you the current state of the grid. You must tell me what your next move is.
2) Your answers must contain the words "MY MOVE:", the instruction of how you want the token to move.
3) You must keep track of the state of the board in order to win the game.

Movement:
1) You are only allowed to move if you have a token on the board.
2) In the beginning of the game, you do not have any tokens on the board.
3) You are only allowed to add your token when I roll a 6. The token is placed on the 1st field on the board.
4) If I do not roll a 6, your token cannot be added to the board.
5) You can only move from left to right.
6) You are only allo